In [1]:
import requests
import pandas as pd
from io import BytesIO
import os
from core.utils import solve_path, solve_dir, check_file_exists

In [2]:
ORIGINAL_DATA_DIR = solve_dir('original_data')
GENERATED_DATA_DIR = solve_dir('generated_data')
GENERATED_DATA_DIR = solve_path('estrutura_pdm', GENERATED_DATA_DIR)
GENERATED_DATA_DIR = solve_dir(GENERATED_DATA_DIR)

In [3]:
link_pdm_final = 'https://www.prefeitura.sp.gov.br/cidade/secretarias/upload/governo/arquivos/pdm-2124/Planilha%20PdM%2021-24%20Dados%20Abertos.xlsx'

In [4]:
def download_xl(link, fname, folder, **read_excel):
    
    fpath = solve_path(fname, parent=folder)
    
    if os.path.exists(fpath):
        return pd.read_excel(fpath)
    
    with requests.get(link) as r:
        data = r.content
    data = BytesIO(data)
    df = pd.read_excel(data, **read_excel)
    df.to_excel(fpath, index=False)
    
    return df

In [5]:
df = download_xl(link_pdm_final, 'pdm_versao_final.xlsx',
                 ORIGINAL_DATA_DIR, skiprows=1)

In [6]:
de_para_metas = {
    'Meta_numero' : 'codigo',
    'Meta_descricao' : 'titulo',
    'Contexto' : 'contexto',
    'Informações Complementares' : 'complemento',
    'Eixo' : 'macrotema',
    'Objetivo Estratégico' : 'tema',
    'Secretaria Responsável' : 'orgao',
    'ODS vinculados' : 'ods',
    'Indicador' : 'indicador'
}

In [7]:
[k for k in df.keys() if k not in de_para_metas]

[]

In [8]:
df = df.rename(de_para_metas, axis=1)

In [9]:
df['pdm_id'] = 1

In [10]:
df['id'] = df.index + 1

In [11]:
def normalize_table(df, col, renamed_col = False):
    
    table = df[[col, 'pdm_id']]
    table = table.drop_duplicates().reset_index(drop=True)
    table['id'] = table.index+1
    
    mapped = table[['id', col]].to_dict(orient='dict')[col]
    #key+1 para coluna id começar em 1
    mapped = {val : key+1 for key, val in mapped.items()}
    
    df[col+'_id'] = df[col].apply(lambda x: mapped[x])
    df.drop(col, axis=1, inplace=True)
    
    if renamed_col:
        table.rename({col : renamed_col}, axis=1, inplace=True)
    
    
    return table

In [12]:
macrotemas = normalize_table(df, 'macrotema', 'descricao')

In [13]:
macrotemas

,descricao,pdm_id,id
0,SP Justa e Inclusiva,1,1
1,SP Segura e Bem Cuidada,1,2
2,SP Ágil,1,3
3,SP Inovadora e Criativa,1,4
4,SP Global e Sustentável,1,5
5,SP Eficiente,1,6


In [14]:
temas = normalize_table(df, 'tema', 'descricao')

In [15]:
temas.head()

,descricao,pdm_id,id
0,Reduzir a pobreza e ampliar o acesso a direito...,1,1
1,Garantir à população atendimento integral em s...,1,2
2,Garantir a proteção integral e o pleno desenvo...,1,3
3,"Promover o acesso à moradia, à urbanização e à...",1,4
4,Garantir à população idosa o pleno exercício d...,1,5


In [16]:
#não usamos subtema
df['subtema_id'] = None

In [17]:
def extrair_orgaos(df):
    
    orgaos = df['orgao'].unique()
    orgaos_unicos = set()
    
    for item in orgaos:
        splited = item.split(',')
        for sigla in splited:
            sigla = sigla.strip()
            orgaos_unicos.add(sigla)
            
    return orgaos_unicos

In [18]:
orgaos_unicos = extrair_orgaos(df)

In [19]:
orgaos_table = pd.read_csv(solve_path('orgaos.csv',
                                      solve_path('estrutura_administrativa_simplificada',
                                                'generated_data')),
                          sep=';', encoding='latin-1')

In [20]:
orgaos_table.sample(5)

,id,tipo_orgao_id,sigla,descricao
30,31,1,SMUL,Secretaria Municipal de Urbanismo e Licenciamento
74,11,3,SPOBRAS,São Paulo Obras
13,14,1,SMS,Secretaria Municipal da Saúde
20,21,1,SEME,Secretaria Municipal de Esportes e Lazer
47,16,2,SUBLA,Subprefeitura Lapa


In [21]:
for sigla in orgaos_unicos:
    if sigla not in orgaos_table['sigla'].unique():
        print(sigla)
else:
    print('Todos os órgãos já estão na tabela')

Todos os órgãos já estão na tabela


In [22]:
def sigla_in_orgaos(sigla_alvo, orgao_cell):
    
    splited = orgao_cell.split(',')
    
    for sigla in splited:
        sigla = sigla.strip()
        if sigla==sigla_alvo:
            return True
    else:
        return False

def cross_table_orgaos(df, orgaos_table):
    
    mapped = orgaos_table.to_dict()['sigla']
    #key+1 para indexar em 1 não 0
    mapped = {val : key+1 for key, val in mapped.items()}
    
    data = []
    id_ = 1
    for i, row in df.iterrows():
        orgao_cell = row['orgao']
        meta_id = row['id']
        for orgao_sigla, orgao_id in mapped.items():
            if sigla_in_orgaos(orgao_sigla, orgao_cell):
                #colocando False para todos no orgao responsavel
                #depois equipe CP atualiza essa info
                data_row = (id_, meta_id, orgao_id, False)
                data.append(data_row)
                id_+=1
    df.drop('orgao', axis=1, inplace=True)
    
    return pd.DataFrame(data, columns = ('id', 'meta_id', 
                                         'orgao_id', 'orgao_responsavel'))

In [23]:
meta_orgao = cross_table_orgaos(df, orgaos_table)

In [24]:
meta_orgao.sample(5)

,id,meta_id,orgao_id,orgao_responsavel
89,90,61,18,False
55,56,35,15,False
101,102,68,32,False
75,76,53,17,False
41,42,23,20,False


In [25]:
def save_df(df, fname, folder = GENERATED_DATA_DIR):
    
    fname = solve_path(fname, folder)
    
    try:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='latin-1', index=False)
    except UnicodeEncodeError:
        df.to_csv(fname, sep =';', decimal=',', 
                  encoding='utf-8', index=False)

In [26]:
df.rename({'indicador' : 'indicador_temp',
          'ods' : 'ods_temp'}, axis=1, inplace=True)

In [27]:
df['inativa'] = False
df['status'] = None
df['motivo_inativacao'] = None
df['data_inativacao'] = None

In [28]:
df.fillna(value='NULL', inplace=True)

In [29]:
df.keys()

Index(['codigo', 'titulo', 'indicador_temp', 'contexto', 'complemento',
       'ods_temp', 'pdm_id', 'id', 'macrotema_id', 'tema_id', 'subtema_id',
       'inativa', 'status', 'motivo_inativacao', 'data_inativacao'],
      dtype='object')

In [30]:
col_order = [
             'id', 
             'pdm_id',
             'codigo', 
             'titulo', 
             'contexto',
             'complemento',
             'macrotema_id', 
             'tema_id', 
             'subtema_id',
             'status', 
             'inativa', 
             'motivo_inativacao', 
             'data_inativacao',
             'indicador_temp',
             'ods_temp',
            ]

In [31]:
df = df[col_order]

In [32]:
save_df(df, 'Meta.csv')

In [33]:
save_df(macrotemas, 'macro_tema.csv')

In [34]:
save_df(temas, 'tema.csv')

In [35]:
save_df(meta_orgao, 'Meta_Orgao.csv')